In [ ]:
! pip install -q -U trax

In [ ]:
import os
import trax
import trax.fastmath.numpy as np
import pickle
import numpy
import random as rnd

from trax import fastmath
from trax import layers as tl

#trax.supervised.trainer_lib.init_random_number_generators(32)
rnd.seed(32)

Importing the data

In [ ]:
dirname = '/content/data/'
lines = []
for filename in os.listdir(dirname):
  with open(os.path.join(dirname, filename)) as files:
    for line in files:
      pure_line = line.strip()
      if pure_line:
        lines.append(pure_line)

In [ ]:
n_lines = len(lines)
print(f'number of lines: {n_lines}')
print(f'Sample line at position 0 {lines[0]}')
print(f'Sample line at position 999 {lines[999]}')

number of lines: 125097
Sample line at position 0 HAMLET
Sample line at position 999 That you, at such times seeing me, never shall,


In [ ]:
for i, line in enumerate(lines):
  #convert all to lowercase
  lines[i] = line.lower()

print(f"Number of lines: {n_lines}")
print(f"Sample line at position 0 {lines[0]}")
print(f"Sample line at position 999 {lines[999]}")

Number of lines: 125097
Sample line at position 0 hamlet
Sample line at position 999 that you, at such times seeing me, never shall,


In [ ]:
eval_lines = lines[-1000:] # Create a holdout validation set
lines = lines[:-1000] # Leave the rest for training

print(f"Number of lines for training: {len(lines)}")
print(f"Number of lines for validation: {len(eval_lines)}")

Number of lines for training: 124097
Number of lines for validation: 1000


In [ ]:
def line_to_tensor(line, EOS_int=1):
  tensor = []
  for c in line:
    tensor.append(ord(c))
  
  tensor.append(EOS_int)
  return tensor

In [ ]:
line_to_tensor('abc xyz')

[97, 98, 99, 32, 120, 121, 122, 1]

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: data_generator
def data_generator(batch_size, max_length, data_lines, line_to_tensor=line_to_tensor, shuffle=True):
    """Generator function that yields batches of data

    Args:
        batch_size (int): number of examples (in this case, sentences) per batch.
        max_length (int): maximum length of the output tensor.
        NOTE: max_length includes the end-of-sentence character that will be added
                to the tensor.  
                Keep in mind that the length of the tensor is always 1 + the length
                of the original line of characters.
        data_lines (list): list of the sentences to group into batches.
        line_to_tensor (function, optional): function that converts line to tensor. Defaults to line_to_tensor.
        shuffle (bool, optional): True if the generator should generate random batches of data. Defaults to True.

    Yields:
        tuple: two copies of the batch (jax.interpreters.xla.DeviceArray) and mask (jax.interpreters.xla.DeviceArray).
        NOTE: jax.interpreters.xla.DeviceArray is trax's version of numpy.ndarray
    """
    # initialize the index that points to the current position in the lines index array
    index = 0
    
    # initialize the list that will contain the current batch
    cur_batch = []
    
    # count the number of lines in data_lines
    num_lines = len(data_lines)
    
    # create an array with the indexes of data_lines that can be shuffled
    lines_index = [*range(num_lines)]
    
    # shuffle line indexes if shuffle is set to True
    if shuffle:
        rnd.shuffle(lines_index)
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    while True:
        
        # if the index is greater or equal than to the number of lines in data_lines
        if index >= num_lines:
            # then reset the index to 0
            index = 0
            # shuffle line indexes if shuffle is set to True
            if shuffle:
                rnd.shuffle(lines_index)
            
        # get a line at the `lines_index[index]` position in data_lines
        line = data_lines[lines_index[index]]
        
        # if the length of the line is less than max_length
        if len(line) < max_length:
            # append the line to the current batch
            cur_batch.append(line)
            
        # increment the index by one
        index += 1
        
        # if the current batch is now equal to the desired batch size
        if len(cur_batch) == batch_size:
            
            batch = []
            mask = []
            
            # go through each line (li) in cur_batch
            for li in cur_batch:
                # convert the line (li) to a tensor of integers
                tensor = line_to_tensor(li)
                
                # Create a list of zeros to represent the padding
                # so that the tensor plus padding will have length `max_length`
                pad = [0] * (max_length - len(tensor))
                
                # combine the tensor plus pad
                tensor_pad = tensor + pad
                
                # append the padded tensor to the batch
                batch.append(tensor_pad)

                # A mask for  tensor_pad is 1 wherever tensor_pad is not
                # 0 and 0 wherever tensor_pad is 0, i.e. if tensor_pad is
                # [1, 2, 3, 0, 0, 0] then example_mask should be
                # [1, 1, 1, 0, 0, 0]
                # Hint: Use a list comprehension for this
                example_mask = [0 if t == 0 else 1 for t in tensor_pad]
                mask.append(example_mask)
               
            # convert the batch (data type list) to a trax's numpy array
            batch_np_arr = np.array(batch)
            mask_np_arr = np.array(mask)
            
            ### END CODE HERE ##
            
            # Yield two copies of the batch and mask.
            yield batch_np_arr, batch_np_arr, mask_np_arr
            
            # reset the current batch to an empty list
            cur_batch = []
            

In [ ]:
import itertools

infinite_data_generator = itertools.cycle(
    data_generator(batch_size=2, max_length=10, data_lines=tmp_lines))

In [ ]:
ten_lines = [next(infinite_data_generator) for _ in range(10)]
print(len(ten_lines))

10


In [ ]:
def GRULM(vocab_size=256, d_model=512, n_layers=2, mode='train'):
    model = tl.Serial(
      tl.ShiftRight(mode=mode), # Stack the ShiftRight layer
      tl.Embedding(vocab_size=vocab_size, d_feature=d_model), # Stack the embedding layer
      [tl.GRU(n_units=d_model) for _ in range(n_layers)], # Stack GRU layers of d_model units keeping n_layer parameter in mind (use list comprehension syntax)
      tl.Dense(n_units=vocab_size), # Dense layer
      tl.LogSoftmax() # Log Softmax
    )
    return model


In [ ]:
model = GRULM()
print(model)

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_256_512
  GRU_512
  GRU_512
  Dense_256
  LogSoftmax
]


In [ ]:
batch_size = 32
max_length = 64

In [ ]:
def n_used_lines(lines, max_length):
    '''
    Args: 
    lines: all lines of text an array of lines
    max_length - max_length of a line in order to be considered an int
    output_dir - folder to save your file an int
    Return:
    number of efective examples
    '''

    n_lines = 0
    for l in lines:
        if len(l) <= max_length:
            n_lines += 1
    return n_lines

num_used_lines = n_used_lines(lines, 32)
print('Number of used lines from the dataset:', num_used_lines)
print('Batch size (a power of 2):', int(batch_size))
steps_per_epoch = int(num_used_lines/batch_size)
print('Number of steps to cover one epoch:', steps_per_epoch)

Number of used lines from the dataset: 25927
Batch size (a power of 2): 32
Number of steps to cover one epoch: 810


In [ ]:
from trax.supervised import training
def train_model(model, data_generator, batch_size=32, max_length=64, eval_lines=eval_lines, n_steps=100, output_dir='/content/model2'):
  bare_train_generator = data_generator(batch_size, max_length, data_lines=lines)
  infinite_train_generator = itertools.cycle(bare_train_generator)

  bare_eval_generator = data_generator(batch_size, max_length, data_lines=eval_lines)
  infinite_eval_generator = itertools.cycle(bare_eval_generator)

  train_task = training.TrainTask(
      labeled_data = infinite_train_generator,
      loss_layer = tl.CrossEntropyLoss(),
      optimizer = trax.optimizers.Adam(0.0005)
  )

  eval_task = training.EvalTask(
      labeled_data = infinite_eval_generator,
      metrics = [tl.CrossEntropyLoss(), tl.Accuracy()],
      n_eval_batches=3
  )

  training_loop = training.Loop(model,
                                train_task,
                                eval_tasks=[eval_task],
                                output_dir=output_dir)
  training_loop.run(n_steps=n_steps)
  return training_loop

In [ ]:
train_model(GRULM(), data_generator)

/usr/local/lib/python3.9/dist-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 3411200
Step      1: Ran 1 train steps in 7.01 secs
Step      1: train CrossEntropyLoss |  5.54509401


/usr/local/lib/python3.9/dist-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  5.54105727
Step      1: eval          Accuracy |  0.14964740

Step    100: Ran 99 train steps in 177.33 secs
Step    100: train CrossEntropyLoss |  3.37116504
Step    100: eval  CrossEntropyLoss |  2.91466570
Step    100: eval          Accuracy |  0.18991873
